# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui

sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape

# Não há duplicados

sinasc[['ESTCIVMAE', 'CONSULTAS','QTDFILVIVO', 'APGAR1', 'APGAR5']].isna().sum()

(27028, 69)


ESTCIVMAE      317
CONSULTAS        0
QTDFILVIVO    1573
APGAR1          96
APGAR5         103
dtype: int64

In [2]:
# 2) seu código aqui Conte o número de valores missing por variável.

sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui 

cnt_missing = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].isna().sum()
print(cnt_missing)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [4]:
# 4) seu código aqui 

print(sinasc.shape)
print(cnt_missing)

(27028, 69)
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [5]:
sinasc.drop(['APGAR5'], axis=1, inplace=True)
print(sinasc.shape)
print(cnt_missing)

(27028, 68)
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [6]:
sinasc['QTDFILVIVO'].unique()

array([ 0.,  1.,  2.,  3., nan,  4.,  6.,  5.,  7., 12.,  9., 11.,  8.,
       30., 10., 14.])

In [7]:
# 5) seu código aqui

sinasc['ESTCIVMAE'].replace([9],0, inplace=True)
sinasc['CONSULTAS'].replace([9],0, inplace=True)

In [8]:
# 6) Seu código aqui

sinasc['QTDFILVIVO'].fillna(0, inplace=True)

In [9]:
# 7) seu código aqui
# Justificativa: os 'Na' foram substituídos por '0' porque, para o banco de dados em questão lidar com numerais (int ou float) 
# permitirá extrair informações mais precisas. Das colunas referids no item 3 somente 'ESTCIVMAE', 'ESCMAE','QTDFILVIVO', 
#'GESTACAO' e 'GRAVIDEZ' possuem dados 'missing' e destas, só 'ESTCIVMAE' e 'QTDFILVIVO' serão impacatdas pela substituição,
# evitando o enviesamento.

sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS']].fillna(0).inplace=True

In [10]:
# 8) seu código aqui

sinasc.loc[(sinasc['APGAR1']>= 8) & (sinasc['APGAR1']<= 10), 'APGAR_CAT'] = 'normal'
sinasc.loc[(sinasc['APGAR1']>= 6) & (sinasc['APGAR1']<= 7), 'APGAR_CAT'] = 'asfixia leve'
sinasc.loc[(sinasc['APGAR1']>= 4) & (sinasc['APGAR1']<= 5), 'APGAR_CAT'] = 'asfixia moderada'
sinasc.loc[(sinasc['APGAR1']>= 0) & (sinasc['APGAR1']<= 3), 'APGAR_CAT'] = 'asfixia severa'

In [11]:
sinasc[['APGAR1', 'APGAR_CAT']].head(60)

,APGAR1,APGAR_CAT
0,9.0,normal
1,8.0,normal
2,8.0,normal
3,9.0,normal
4,8.0,normal
5,9.0,normal
6,9.0,normal
7,9.0,normal
8,8.0,normal
9,9.0,normal


In [12]:
# 9) seu código aqui 

sinasc.rename({'LOCNASC': 'Local_Nascimento', 'IDADEMAE': 'Idade_Mãe', 'ESTCIVMAE': 'Estaco_Civil_Mãe', 
                'ESCMAE': 'Escolaridade_Mãe', 'QTDFILVIVO': 'Quantidade_Filho_Vivo', 'GESTACAO': 'Gestação',
                'GRAVIDEZ': 'Gravidez', 'CONSULTAS': 'Consultas'}, axis=1, inplace=True)

In [13]:
sinasc.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'Local_Nascimento', 'Idade_Mãe',
       'Estaco_Civil_Mãe', 'Escolaridade_Mãe', 'CODOCUPMAE',
       'Quantidade_Filho_Vivo', 'QTDFILMORT', 'CODMUNRES', 'Gestação',
       'Gravidez', 'PARTO', 'Consultas', 'DTNASC', 'HORANASC', 'SEXO',
       'APGAR1', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO', 'CODANOMAL',
       'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA', 'DTRECORIGA',
       'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010', 'SERIESCMAE',
       'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR', 'QTDPARTCES',
       'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM', 'CONSPRENAT',
       'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO', 'TPNASCASSI',
       'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1', 'STDNEPIDEM',
       'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE', 'KOTELCHUCK',
       'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome', 'munResUf',
       'munResLat', 'munResLon', 'munResAlt', 'm